# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastai.vision.all import *
from datasets import load_dataset

ds = load_dataset('competitions/aiornot')

#path = Path('/home/gegg00/Desktop/aiornot/')
#df = pd.read_csv(path/'train.csv')

In [ ]:
ds['train'][0:5]['image']

In [ ]:
import cv2
import albumentations
import numpy as np

transform = albumentations.Compose([
    albumentations.RandomCrop(width=256, height=256),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.RandomBrightnessContrast(p=0.2),
])

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [
        transform(image=np.array(image))["image"] for image in examples["image"]
    ]

    return examples

In [ ]:
ds.set_transform(transforms)

In [ ]:
plt.imshow(ds['train'][2]['pixel_values'])

In [ ]:
model_checkpoint = "microsoft/swin-tiny-patch4-window7-224" # pre-trained model from which to fine-tune
batch_size = 32 # batch size for training and evaluation

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("image_classification_notebook", framework="pytorch")

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

In [ ]:
from datasets import load_dataset

dataset = load_dataset('competitions/aiornot')


In [ ]:
dataset

In [ ]:
example = dataset['train'][10]
example

In [ ]:
dataset['train'].features

In [ ]:
example['image']

In [ ]:
example['image'].resize((200,200))

In [ ]:
example['label']

In [ ]:
dataset["train"].features["label"]

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

In [ ]:
(feature_extractor.size['height'], feature_extractor.size['width'])

In [ ]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose(
        [
            RandomResizedCrop(feature_extractor.size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize((feature_extractor.size['height'], feature_extractor.size['width'])),
            CenterCrop(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

    

In [ ]:
# split up training into training + validation
splits = dataset["train"].train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

In [ ]:
train_ds

In [ ]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint, 
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

In [ ]:
#| export
def train(arch, size, accum=1, finetune=True, epochs=12):
    dls = DataBlock(
        blocks=(ImageBlock, CategoryBlock), 
        get_items=get_images, 
        splitter=RandomSplitter(valid_pct=0.2, seed=42),
        get_y=get_label,
        item_tfms=[Resize(size, method='squish')],
        batch_tfms=aug_transforms(size=size, min_scale=0.75)
    ).dataloaders(path, bs=64//accum)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    if finetune:
        learn.fine_tune(epochs, 0.01)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, 0.01)

In [ ]:
#| export
from fastcore.script import *
@call_parse
def main(arch:str,   #Model to use from timm 
         size:int,  #size of the images
         accum:int, #size of the batch
         epochs:int, #number of epochs
         finetune:bool):  #Fine tune model
    print(train(arch, size, accum, finetune, epochs=epochs))
    #tst_files = get_image_files(path/'data/test').sorted()
    #tst_dl = dls.test_dl(tst_files)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()